## Início do algoritmo
O primeiro passo é o mais simples. Apenas vou ler os dados utilizando a biblioteca `pandas`.

In [1]:
import pandas as pd

#dados dos filmes e notas
filmes = pd.read_csv('../data/movies.csv')
notas = pd.read_csv('../data/ratings.csv')

In [ ]:
# Pré visualização dos filmes
print(filmes.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [12]:
# Pré processamento das notas
print(notas.head())

   userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580


## Pré processamento dos dados
Preciso limpar os dados antes de proceguir. Nesse caso, retirar valores vazios utilizando o método `dropna()` da biblioteca `pandas`.

In [3]:
# Etapa de verificação primária

#Valores nulos em notas:
print(notas.isnull().sum())
#Valores nulos em filmes:
print(filmes.isnull().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
movieId    0
title      0
genres     0
dtype: int64


Não encontrei **valores ausentes**, logo não vou precisar utilizar o `dropana()` do `pandas`.
Segue um exemplo se caso tivesse **valores vazios** nos dados:

In [4]:
#Remoção de valores nulos
notas = notas.dropna()
filmes = filmes.dropna()

## Implementação do algoritmo
Vou utilizar a biblioteca `pandas` para realizar o calculo de média de forma eficiente. Vou unir o identificador de cada filme `movieId` aos valores de avaliação `rating` utilizando o método `groupby` do `pandas`. Para realizar o calculo de média de forma eficiente, vou utilizar o método `.mean()` do pandas.

In [6]:
#Cálculo da média de avaliações por filme

num_media = notas.groupby('movieId')['rating'].mean()
print(num_media)

movieId
1         3.921240
2         3.211977
3         3.151040
4         2.861393
5         3.064592
            ...   
131254    4.000000
131256    4.000000
131258    2.500000
131260    3.000000
131262    4.000000
Name: rating, Length: 26744, dtype: float64


## Possíveis solução para resolução do problema

Para recomendar filmes a um usuário, foi criada a função `filmes_recomendados()`.

* Primeiro, ela descobre quais filmes o usuário já assistiu, usando o `userId` para filtrar suas avaliações. Esses filmes ficam guardados na lista `filmes_assistidos`.
* Depois, a função pega as notas médias de todos os filmes (`media_avaliacoes`) e remove os que já estão na lista de filmes assistidos.
* Os filmes que sobraram são organizados do maior para o menor de acordo com a nota média.
* Em seguida, são escolhidos apenas os **n** melhores (usando o parâmetro `n_recomendacoes`).
* Por fim, a função retorna os detalhes desses filmes, buscando no conjunto de dados de filmes (**movies**) as informações correspondentes.

No teste, essa função recomendou **5 filmes** para o usuário de identificador **1**, e os resultados foram exibidos para conferência.



In [8]:
def filmes_recomendados(user_id, n_recomendacoes=5):
    # Descobre quais filmes o usuário já assistiu
    notas_usuario = notas[notas['userId'] == user_id]
    # Lista de filmes assistidos
    filmes_assistidos = notas_usuario['movieId'].tolist()
    # Filtra as médias para remover os filmes já assistidos
    recomendacoes = num_media[~num_media.index.isin(filmes_assistidos)]
    # Seleciona os n melhores filmes
    filmes_recomendados = recomendacoes.nlargest(n_recomendacoes).index.tolist()
    # Retorna os detalhes dos filmes recomendados
    return filmes[filmes['movieId'].isin(filmes_recomendados)]

# Teste da função
chamada_filmes_recomendados = filmes_recomendados(user_id=1, n_recomendacoes=5)
print("Filmes recomendados para o usuário 1:")
# Exibe os filmes recomendados
print(chamada_filmes_recomendados)

Filmes recomendados para o usuário 1:
       movieId                                              title  \
9007     26718      Life On A String (Bian chang Bian Zou) (1991)   
9561     27914  Hijacking Catastrophe: 9/11, Fear & the Sellin...   
9862     32230            Snow Queen, The (Lumikuningatar) (1986)   
10567    40404                                Al otro lado (2004)   
12015    54326                                  Sierra, La (2005)   

                                genres  
9007   Adventure|Drama|Fantasy|Musical  
9561                       Documentary  
9862                  Children|Fantasy  
10567                            Drama  
12015                      Documentary  


In [14]:
def recomendacao_filmes_redefinida(user_id, n_recomendacoes=5, min_num_ratings=50):
    user_ratings = notas[notas['userId'] == user_id]
    watched_movies = user_ratings['movieId'].tolist()

    # Filmes não assistidos
    unwatched_movies = num_media[~num_media.index.isin(watched_movies)]
    print(f'Total de filmes não assistidos: {len(unwatched_movies)}')

    # Filmes populares
    popular_movies = notas['movieId'].value_counts()[notas['movieId'].value_counts() >= min_num_ratings].index
    unwatched_popular_movies = unwatched_movies[unwatched_movies.index.isin(popular_movies)]

    # Recomendações
    recommended_movies = unwatched_popular_movies.sort_values(ascending=False).head(n_recomendacoes).index.tolist()

    print(f'Total de filmes recomendados: {len(recommended_movies)}')
    return filmes[filmes['movieId'].isin(recommended_movies)]

# chamada correta
resultados = recomendacao_filmes_redefinida(user_id=1, n_recomendacoes=5, min_num_ratings=5)
print("Filmes recomendados para o usuário 1 com a função redefinida:")
print(resultados)


Total de filmes não assistidos: 26569
Total de filmes recomendados: 5
Filmes recomendados para o usuário 1 com a função redefinida:
       movieId                                          title           genres
15701    79842                                For Neda (2010)      Documentary
17605    88570                                 Welfare (1975)      Documentary
19549    96845                 Two Rabbits (2 Coelhos) (2012)           Action
23860   113315  Zero Motivation (Efes beyahasei enosh) (2014)     Comedy|Drama
24495   115998                           The Guardians (2012)  Action|Thriller
